In [7]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import backend as K
%matplotlib qt

Inicialización de los parámetros

In [33]:
seed=6                           # for reproducibility 
np.random.seed(seed)

w1 = np.random.normal(loc=0, scale=1, size=(3,2)) #Capa 1 afectada por capa 0
w2 = np.random.normal(loc=0, scale=1, size=(3,)) #Capa 2 afectada por capa 1

Patrones de aprendizaje

In [34]:
ntrain = 4 #Cantidad de patrones para entrenar
x_train = np.zeros((ntrain, 3), dtype = np.float32) #Entradas
y_train = np.zeros((ntrain, 1), dtype = np.float32) #Salidas

x_train[0][0] = 1
x_train[0][1] = 1
y_train[0] = 1

x_train[1][0] = 1
x_train[1][1] = -1
y_train[1] = -1

x_train[2][0] = -1
x_train[2][1] = 1
y_train[2] = -1

x_train[3][0] = -1
x_train[3][1] = -1
y_train[3] = 1

#Neurona de bias en la capa de entrada
for i in range(ntrain):
    x_train[i][2] = -1

Funciones

In [35]:
def MSE(y_esp, y_obt):
    '''
    Recibe el vector de patrones de salidas esperadas y obtenidas (escalares) y calcula el error cuadrático
    '''
    e = 0
    for i in range(len(y_esp)):
        for j in range(len(y_esp[0])):
            e += (y_esp[i][j] - y_obt[i][j])**2
    
    e = 0.5*e

    return e

def g(x):
    return np.tanh(x)

def g_pr(x):
    y = 1 - np.tanh(x)**2
    return y

def v1_accuracy(y_true, y_pred):
    return np.mean(K.mean(K.equal(y_true, K.round(y_pred)), axis=-1))

Variables para el loop de aprendizaje

In [36]:
r = 6 #Cantidad de realizaciones
lrs = [0.01, 0.05, 0.1, 0.5, 1, 5] #learning rates
epocas = 1000


Vectores para luego analizar el desempeño promedio de la red

In [37]:
accuracy = np.zeros(shape=(epocas, r)) #Accuracy de la neurona
mse = np.zeros(shape=(epocas, r))

### Loop de aprendizaje

In [38]:
for i in range(len(lrs)):
    lr = lrs[i]

    #Módulos de las correcciones
    deltas1 = np.zeros(epocas) #Módulo del vector deltaW1
    deltas2 = np.zeros(epocas)  #Módulo del vector deltaW2

    #Salidas de la red para cada época
    salidas = np.zeros((ntrain, 1), dtype = np.float32) #Salidas #Registro las salidas predichas y calculo accuracy


    for k in range(epocas):
        deltaW1 = np.zeros(shape=(3,2))
        deltaW2 = np.zeros(shape=(3,))

        for mu in range(ntrain):

        #FORWARD PROPAGATION (entrada)
            #Capa de entrada
            c0 = x_train[mu]
            # print("\n\nEntrada")
            # print(c0)

            #Capa oculta
            h1 = np.dot(c0,w1)
            c1 = g(h1)
            # print("\nCapa oculta")
            # print(c0)
            # print(w1)
            # print(h1)
            # print(c1)

            #Capa de salida
            h2 = np.dot(np.concatenate([c1, [c0[2]]]), w2)
            c2 = g(h2) #output
            salidas[mu] = c2
            # print("\nSalida")
            # print(np.concatenate([c1, [c0[2]]]))
            # print(w2)
            # print(h2)
            # print(c2)


        #BACK PROPAGATION (delta)    
            delta2 = (y_train[mu] - c2)*g_pr(h2)
            # print("\nBack propagation")
            # print(delta2)
            # print(w2[:-1]) 
            # print(g_pr(h1))

            delta1 = delta2*w2[:-1]*g_pr(h1) #No tengo que backpropagar el delta a la neurona del bias (omite el último elemento de w2)
            # print(delta1)
            #break

        #FORWARD PROPAGATION (correccion)
            deltaW1 += lr * c0[:, np.newaxis] * delta1 #doy vuelta c0 para obtener el producto vectorial
            # print("\nCorrecciones 1")
            # print(lr)
            # print(c0[:, np.newaxis])
            # print(delta1)
            # print(deltaW1)
            #break

            deltaW2 += lr * np.concatenate([c1, [c0[2]]]) * delta2
            # print("\nCorrecciones 2")
            # print(lr)
            # print(np.concatenate([c1, [c0[2]]]))
            # print(delta2)
            # print(deltaW2)

        w1 += deltaW1
        w2 += deltaW2

        deltas1[k] = np.linalg.norm(deltaW1)
        deltas2[k] = np.linalg.norm(deltaW2)
        mse[k, i] = MSE(y_train, salidas)
        accuracy[k, i] = v1_accuracy(y_train, salidas)


    # print("Matrices de pesos finales")
    # print(w1)
    # print(w2)
    # print(mse[-1, i])


        

#### Gráficos

In [39]:
for i in range(len(lrs)):
    plt.semilogy(range(1, epocas+1), mse[:,i], label = str(lrs[i]))

plt.xlabel("Epoca")
plt.ylabel("MSE")
plt.legend()
plt.show()


#### Guardo datos

In [40]:
import os
os.chdir("D:\IB/7mo cuatri\Redes neuronales\Practicas\Practica4")
f_out = open("Ejercicio1/1a.txt", "w")
f_out.write("ep loss loss_stdv acc acc_stdv\n")
for i in range(epocas):
    f_out.write(str(i+1) + " " + str(mse_mean[i]) + " " + str(mse_std[i]) +  " " + str(acc_mean[i]) + " " + str(acc_std[i]) + "\n")
f_out.close()

NameError: name 'mse_mean' is not defined